## Benchmarking and Calibrating Quantum Devices with Qiskit Experiments: Part 2

Helena Zhang, IBM Quantum (helena.zhang@ibm.com)

IEEE Quantum Week 2023

We'll be live coding in Qiskit and sending jobs to a real backend.

If you'd like to follow along: sign up on [quantum-computing.ibm.com](https://quantum-computing.ibm.com) and send me your email

## Installation

In [ ]:
!pip install git+https://github.com/Qiskit-Extensions/qiskit-experiments.git@f5c759bd70b875d15183b9d2b8a7374ea0f5cad3

In [ ]:
!python -m pip list | grep qiskit

## Running on a real backend

In [ ]:
import qiskit_ibm_provider.jupyter
from qiskit_ibm_provider import IBMProvider

# If you haven't yet saved the token to your account, do this:
# IBMProvider.save_account(token=<INSERT_IBM_QUANTUM_TOKEN>)

provider = IBMProvider(instance="ibm-q-community/ieee-tutorial/main")
backend_real = provider.get_backend('ibm_cairo')

In [ ]:
# alternatively, without the widget: 
# from qiskit.visualization import plot_error_map
# plot_error_map(backend)

backend_real

Let's write an experiment class to generate the GHZ state.

To write your own experiment subclassing from `BaseExperiment`:

- Call the `__init__()` method during the subclass
  constructor with a list of physical qubits and initialization options.
  
- Implement the abstract `circuits()` method.
  This should return a list of `QuantumCircuit` objects defining
  the experiment payload. Each circuit can have its own metadata in addition to experiment metadata


- Experiment and execution options can be overridden as needed.

## Custom experiment template

In [ ]:
from qiskit.circuit import QuantumCircuit
from typing import List, Optional, Sequence
from qiskit.providers.backend import Backend
from qiskit_experiments.framework import BaseExperiment, Options

class CustomExperiment(BaseExperiment):
    """Custom experiment class template."""

    def __init__(self, 
                 physical_qubits: Sequence[int], 
                 backend: Optional[Backend] = None):
        """Initialize the experiment."""
        super().__init__(physical_qubits, 
                         analysis = CustomAnalysis(),
                         backend = backend)

    def circuits(self) -> List[QuantumCircuit]:
        """Generate the list of circuits to be run."""
        circuits = []
        # Generate circuits and populate metadata here
        for i in loops:
            circ = QuantumCircuit(self.num_qubits)
            circ.metadata = {}
            circuits.append(circ)
        return circuits

    @classmethod
    def _default_experiment_options(cls) -> Options:
        """Set default experiment options here."""
        options = super()._default_experiment_options()
        options.update_options(
            dummy_option = None,
        )
        return options

In [ ]:
from qiskit.circuit import QuantumCircuit
from qiskit.providers.backend import Backend
from qiskit_experiments.framework import BaseExperiment

class GHZExperiment(BaseExperiment):
    """Make a GHZ state."""

    def __init__(self, physical_qubits, measure = True, backend = None):
        """Initialize the experiment."""
        super().__init__(physical_qubits, 
                         backend = backend)
        self.set_experiment_options(measure=measure)
        
    def _default_experiment_options(cls):
        r"""Default values for the rough drag experiment.

        Experiment Options:
            measure (Bool): Whether to measure qubits at the end.
        """
        options = super()._default_experiment_options()
        options.measure = True

        return options
        
    def circuits(self):
        """Generate the list of circuits to be run."""
        circuits = []
        circ = QuantumCircuit(self.num_qubits)
        circ.h(0)
        for i in range(1,self.num_qubits):
            circ.cx(0,i)
        if self.experiment_options.measure:
            circ.measure_all()
        circuits.append(circ)
        return circuits

Instantiate an experiment and inspect the circuits:

In [ ]:
exp = GHZExperiment((1,0,2), True, backend_real)
exp.circuits()[0].draw("mpl")

In [ ]:
exp._transpiled_circuits()[0].draw("mpl", idle_wires=False)

In [ ]:
exp_data=exp.run().block_for_results()

In [ ]:
exp_data.data(0)

How do we analyze the quality of our state?

In [ ]:
import qiskit
from qiskit_experiments.framework import ParallelExperiment
from qiskit_experiments.library import StateTomography

exp = GHZExperiment((1,0,2), False, backend_real)

# QST Experiment
qst_exp = StateTomography(exp.circuits()[0])
qst_data = qst_exp.run(backend_real).block_for_results()

qst_data.analysis_results(dataframe=True)

In [ ]:
state_result = qst_data.analysis_results("state")
print(state_result.value)

In [ ]:
from qiskit.visualization import plot_state_city
plot_state_city(qst_data.analysis_results("state").value, title='Density Matrix')

In [ ]:
starting_fid = qst_data.analysis_results("state_fidelity")
print("State Fidelity = {:.5f}".format(starting_fid.value))

## Tutorial outline

- Overview of quantum characterization and calibration
- Introduction to Qiskit Experiments
- **Calibrations in Qiskit Experiments**
- Writing a custom experiment

Let's calibrate our gates on a real device. To see how to calibrate gates in Qiskit (without Experiments), you can browse the [Qiskit textbook](https://learn.qiskit.org/course/quantum-hardware-pulses/calibrating-qubits-using-qiskit-pulse).

In [ ]:
from qiskit_experiments.calibration_management.calibrations import Calibrations

from qiskit_experiments.calibration_management.basis_gate_library import FixedFrequencyTransmon

library = FixedFrequencyTransmon()
cals = Calibrations.from_backend(backend_real, libraries=[library])

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

columns_to_show = ["parameter", "qubits", "schedule", "value", "date_time"]
pd.DataFrame(**cals.parameters_table())[columns_to_show]

In [ ]:
x_sched = cals.get_schedule('x',(0,))
print(x_sched)
x_sched.draw()

## Frequency calibration

In [ ]:
import numpy as np
from qiskit_experiments.library.calibration.rough_frequency import RoughFrequencyCal

def frequency(qubit):
    freq01_estimate = cals.get_parameter_value(param="drive_freq",qubits=[qubit])
    return np.linspace(freq01_estimate-10e6, freq01_estimate+10e6, 51)

experiments=[]

for qubit in range(10):
    exp = RoughFrequencyCal([qubit], cals, frequency(qubit), backend=backend_real)
    exp.set_experiment_options(amp=0.004)
    experiments.append(exp)

spec_cal = ParallelExperiment(experiments, flatten_results=True)
spec_cal.set_run_options(meas_level=1, meas_return="avg")

In [ ]:
spec_data = spec_cal.run(backend_real).block_for_results()

In [ ]:
spec_data.analysis_results(dataframe=True)

In [ ]:
spec_data.figure(0)

In [ ]:
pd.DataFrame(**cals.parameters_table(parameters=["drive_freq"],qubit_list=[0],most_recent_only=False))[columns_to_show]

## Rough amplitude calibration

In [ ]:
from qiskit_experiments.library import RoughXSXAmplitudeCal
from qiskit_experiments.framework import ParallelExperiment

rough_amp_cal = ParallelExperiment([RoughXSXAmplitudeCal([qubit], cals, backend=backend_real) for qubit in range(10)], flatten_results=True)
rough_amp_cal.set_run_options(meas_level=1, meas_return="single")

In [ ]:
rough_amp_cal_data = rough_amp_cal.run(backend_real).block_for_results()
rough_amp_cal_data.analysis_results(dataframe=True)

In [ ]:
pd.DataFrame(**cals.parameters_table(schedules=["x"],qubit_list=[(),0],most_recent_only=False))[columns_to_show]

# DRAG calibration

A Derivative Removal by Adiabatic Gate (DRAG) pulse is designed to minimize leakage and phase errors to a neighbouring transition. The DRAG pulse is $f(t) = \Omega(t) + i \beta \frac{d}{dt} \Omega(t)$, where $\Omega$ is the envelope of the in-phase component of the pulse, and $\beta$ is the strength of the quadrature, which we refer to as the DRAG parameter and seek to calibrate in this experiment.



In [ ]:
from qiskit_experiments.library import RoughDragCal

def frequency(qubit):
    freq01_estimate = cals.get_parameter_value(param="drive_freq",qubits=[qubit])
    return np.linspace(freq01_estimate-10e6, freq01_estimate+10e6, 51)

experiments=[]

for qubit in range(10):
    exp = RoughDragCal([qubit], cals, backend=backend_real, betas=np.linspace(-20, 20, 25))
    exp.set_experiment_options(reps=[3, 5, 7])
    experiments.append(exp)

drag_cal = ParallelExperiment(experiments, flatten_results=True)
exp.circuits()[0].draw("mpl")

In [ ]:
drag_cal_data = drag_cal.run(backend_real)
drag_cal_data.analysis_results(dataframe=True)

In [ ]:
drag_cal_data.figure(0)

Let's look at the final pulses produced by our calibrations:

In [ ]:
cals.get_inst_map().get("x", 0)

Compare with the starting pulse:

In [ ]:
backend_real.instruction_schedule_map.get("x", 0)

We need to update the `sx` gate with the same parameters.

In [ ]:
cals.get_inst_map().get("sx", 0)

In [ ]:
for qubit in range(10):
    x_beta = cals.get_parameter_value("β", qubit, "x")
    cals.add_parameter_value(x_beta, "β", qubit, "sx")
cals.get_inst_map().get("sx", 0)

Let's try out our new pulses! First, send these calibrations along with our circuits. The Qiskit transpiler will use a custom instruction schedule map if one is provided.

In [ ]:
exp = GHZExperiment((0,1,2), False, backend_real)
qst_exp = StateTomography(exp.circuits()[0])
qst_exp.set_transpile_options(inst_map=cals.get_inst_map())
qst_data = qst_exp.run(backend_real)
qst_data.analysis_results(dataframe=True)

Look at the circuits:

In [ ]:
transpile(qst_exp.circuits()[0], inst_map=cals.get_inst_map(), backend=backend_real)

In [ ]:
qst_exp.circuits()[0].draw()

In [ ]:
transpile(qst_exp.circuits()[0],backend_real).draw()

We can draw the pulse schedule:

In [ ]:
from qiskit import schedule
transpiled = transpile(exp.circuits()[0], backend=backend_real)
schedule(transpiled, backend=backend_real).draw(time_range=(0,1000))

In [ ]:
from qiskit import schedule
transpiled = transpile(exp.circuits()[0], inst_map=cals.get_inst_map(), backend=backend_real)
schedule(transpiled, backend=backend_real).draw(time_range=(0,1000))

Let's test that we're actually doing something by deliberately introducing a bad pulse.

In [ ]:
cals.add_parameter_value(0.05, "amp", 0, "sx")
cals.get_inst_map().get("sx", 0)

In [ ]:
exp = GHZExperiment((0,1,2), False, backend_real)
qst_exp = StateTomography(exp.circuits()[0])
qst_exp.set_transpile_options(inst_map=cals.get_inst_map())
qst_data = qst_exp.run(backend_real).block_for_results()
qst_data.analysis_results(dataframe=True)

## Randomized measurement experiment

Symmetrizing the measurement readout error of a circuit is especially useful in systems 
where readout has an unknown and potentially large bias. We can create a custom experiment to take a circuit as an input and symmetrize
its readout.

## Custom analysis class template

In [ ]:
import matplotlib
from typing import Tuple, List
from qiskit_experiments.framework import (
    BaseAnalysis, 
    Options, 
    ExperimentData, 
    AnalysisResultData
)

class CustomAnalysis(BaseAnalysis):
    """Custom analysis class template."""

    @classmethod
    def _default_options(cls) -> Options:
        """Set default analysis options. Plotting is on by default."""

        options = super()._default_options()
        options.dummy_analysis_option = None
        options.plot = True
        options.ax = None
        return options

    def _run_analysis(
        self, 
        experiment_data: ExperimentData
    ) -> Tuple[List[AnalysisResultData], List["matplotlib.figure.Figure"]]:
        """Run the analysis."""

        # Process the data here

        analysis_results = [
            AnalysisResultData(name="dummy result", value=data)
        ]
        figures = []
        if self.options.plot:
            figures.append(self._plot(data))
        return analysis_results, figures

In [ ]:
from numpy.random import default_rng, Generator
from qiskit import QuantumCircuit
from qiskit_experiments.framework import BaseExperiment
from qiskit.quantum_info import random_pauli_list

class RandomizedMeasurement(BaseExperiment):
    def __init__(
      self,
      circuit,
      measured_qubits=None,
      physical_qubits=None,
      backend=None,
      num_samples=10,
      seed=None
    ):

        if physical_qubits is None:
              physical_qubits = tuple(range(circuit.num_qubits))
        if measured_qubits is None:
              measured_qubits = tuple(range(circuit.num_qubits))

        analysis = RandomizedMeasurementAnalysis()
        super().__init__(physical_qubits, analysis=analysis, backend=backend)

        self._circuit = circuit
        self._measured_qubits = measured_qubits

        self.set_experiment_options(num_samples=num_samples, seed=seed)

    @classmethod
    def _default_experiment_options(cls):
        options = super()._default_experiment_options()
        options.num_samples = None
        options.seed = None
        return options

    def circuits(self):
        circ_nc = self._circuit.num_clbits
        meas_nc = len(self._measured_qubits)
        circ_qubits = list(range(self.num_qubits))
        circ_clbits = list(range(circ_nc))
        meas_qubits = self._measured_qubits
        meas_clbits = list(range(circ_nc, circ_nc + meas_nc))

        num_samples = self.experiment_options.num_samples
        if num_samples is None:
            num_samples = 2 ** self.num_qubits

        seed = self.experiment_options.seed
        if isinstance(seed, Generator):
            rng = seed
        else:
            rng = default_rng(seed)

        paulis = random_pauli_list(meas_nc, size=num_samples, phase=False, seed=rng)

        circuits = []
        orig_metadata = self._circuit.metadata or {}
        for pauli in paulis:
            name = f"{self._circuit.name}_{str(pauli)}"
            circ = QuantumCircuit(
              self.num_qubits, circ_nc + meas_nc,
              name=name
            )
            circ.compose(
              self._circuit, circ_qubits, circ_clbits, inplace=True
            )
            circ.compose(pauli, meas_qubits, inplace=True)
            circ.measure(meas_qubits, meas_clbits)
            circ.metadata = orig_metadata.copy()
            circ.metadata["rm_bits"] = meas_clbits
            circ.metadata["rm_sig"] = pauli.x.astype(int).tolist()

            circuits.append(circ)

    return circuits

from qiskit_experiments.framework import BaseAnalysis, AnalysisResultData

class RandomizedMeasurementAnalysis(BaseAnalysis):
    """Analysis for randomized measurement experiment."""

    # Helper dict to swap a clbit value
    _swap_bit = {"0": "1", "1": "0"}

    def _run_analysis(self, experiment_data):

        combined_counts = {}
        for datum in experiment_data.data():
            counts = datum["counts"]
            num_bits = len(next(iter(counts)))
            metadata = datum["metadata"]
            clbits = metadata["rm_bits"]
            sig = metadata["rm_sig"]
            full_sig = num_bits * [0]
            for bit, val in zip(clbits, sig):
                full_sig[bit] = val
            for key, val in counts.items():
                bitstring = self._swap_bitstring(key, full_sig)
                if bitstring in combined_counts:
                    combined_counts[bitstring] += val
                else:
                    combined_counts[bitstring] = val


        result = AnalysisResultData("counts", combined_counts)
        return [result], []

    @classmethod
    def _swap_bitstring(cls, bitstring, sig):
        """Swap a bitstring based signature to flip bits at."""
        # This is very inefficient but demonstrates the basic idea
        # Really should do with bitwise operations of integer counts rep
        return "".join(reversed(
            [cls._swap_bit[b] if sig[- 1 - i] else b for i, b in enumerate(bitstring)]
        ))

In [ ]:
from qiskit.providers.aer import AerSimulator, noise

backend_ideal = AerSimulator()

# Backend with asymetric readout error
p0g1 = 0.3
p1g0 = 0.05
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_readout_error([[1 - p1g0, p1g0], [p0g1, 1 - p0g1]])
noise_backend = AerSimulator(noise_model=noise_model)

In [ ]:
# Experiment parameters
total_shots = 100000
num_samples = 50
shots = total_shots // num_samples

ghz_circs = GHZExperiment((1,0,2), False, backend_real).circuits()[0]


# Run ideal randomized meas experiment
exp = RandomizedMeasurement(ghz_circs, num_samples=num_samples)
exp.circuits()[0].draw("mpl")

In [ ]:
expdata_ideal = exp.run(AerSimulator(), shots=shots)
counts_ideal = expdata_ideal.analysis_results("counts").value
print(counts_ideal)

In [ ]:
# Run noisy randomized meas experiment with readout error
expdata_noise = exp.run(noise_backend, shots=shots)
counts_noise = expdata_noise.analysis_results("counts").value

# Run noisy simulation of the original circuit without randomization
meas_circ = qc.copy()
meas_circ.measure_all()
result = noise_backend.run(meas_circ, shots=total_shots).result()
counts_direct = result.get_counts(0)

from qiskit.visualization import plot_histogram

# Plot counts, ideally randomized one should be more symmetric in noise
# than direct one with asymmetric readout error
plot_histogram([counts_ideal, counts_direct, counts_noise],
            legend=["Ideal",
                    "Asymmetric meas error (Direct)",
                    "Asymmetric meas error (Randomized)"])

## Upcoming features

- Separation of execution and experiment: https://github.com/Qiskit/RFCs/blob/master/0014-overhaul-qiskit-experiments.md
- Integration with Qiskit Runtime primitives

## Takeaways

- Quantum hardware and the needs of users are ever-evolving
- Characterization and calibration are crucial tasks for large-scale processors
- Quantum software should be designed with scalability and usability in mind 

## Acknowledgements

Thanks to many people who have made Qiskit Experiments possible:
- Naoki Kanazawa, Daniel J. Egger, Yael Ben-Haim, Will Shanks, Gadi Aleksandrowicz, Chris Wood, Eli Arbel, Dekel Meirom, Toshinari Itoko, Merav Aharoni, Itamar Goldman, Conrad Haupt, and others
- The Qiskit team and community

You can join us at
- GitHub: https://github.com/Qiskit/qiskit-experiments
- Slack: The public Qiskit Slack https://qisk.it/join-slack #experiments